In [ ]:
!pip install -q gwpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 72.7 MB/s eta 0:00:00


In [ ]:
%%capture
!pip install datasets
!pip install transformers
!pip install git+https://github.com/ernlavr/OpenAttack.git
!pip install nltk
!pip install pytorch-crf
!pip install language-tool-python

In [ ]:
%%capture
import OpenAttack
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import datasets
import transformers
import torch
import random
from importlib import reload

def enforce_reproducibility(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)
    np.random.seed(seed)
enforce_reproducibility()

In [ ]:
def dataset_mapping(x):
    premise = x['premise']
    hypothesis = x['hypothesis']
    label = x['label']  
    return {
        "x": premise + " " + hypothesis,
        "y": label,
    }

def balance_dataset(ds, numSamples=-1):
    """
    Balances the dataset by removing samples
    :param ds: The dataset
    :param numSamples: The number of samples to keep per label
    :return: The balanced indices
    """
    # Get the number of samples for each label
    dss = ds[:]
    labels = dss["fever_gold_label"]
    if numSamples == -1:
        numSamples = len(labels)
        unique, counts = np.unique(labels, return_counts=True)
        counts = np.roll(counts, 1)
        unique = np.roll(unique, 1)
        numSamples = min(counts)

    # get indices of ds elements where ds['label'] is 0
    arr = dss['label']
    arr = np.array(arr)
    indicesSup = np.where(arr == 0)[0][:numSamples]
    indicesNei = np.where(arr == 1)[0][:numSamples]
    indicesRef = np.where(arr == 2)[0][:numSamples]

    # combine the indices
    indices = np.sort((np.concatenate((indicesSup, indicesNei, indicesRef))))
    indices = indices.tolist()
    # get a subset of the dataset
    return indices

print("New Attacker")
print("Build model")
HG_MODEL_NAME = "ernlavr/destilbert_uncased_fever_nli"
HG_DATASET = "pietrolesci/nli_fever"

New Attacker
Build model


In [ ]:
%%capture

# Load model, tokenizer and create an OpenAttack classifier
tokenizer = transformers.AutoTokenizer.from_pretrained(HG_MODEL_NAME)
model = transformers.AutoModelForSequenceClassification.from_pretrained(HG_MODEL_NAME, output_hidden_states=True)
clsf = OpenAttack.classifiers.TransformersClassifier(model, tokenizer=tokenizer, embedding_layer=model.base_model.embeddings.word_embeddings)

In [ ]:
%%capture
# Load, map and balance the dataset as per fine-tuning
ds = datasets.load_dataset(HG_DATASET)
dataset = ds['dev'].map(function=dataset_mapping)
dsIndices = balance_dataset(dataset, 30)
devSet = torch.utils.data.Subset(dataset, dsIndices)

## Certificate Issues
Below is the loop for using UAT attack. There seems to be a problem with the certificates so if you need to re-run this then copy+paste the bottom code in zip_downloader.py (access it from the callstack)


```
/usr/local/lib/python3.8/dist-packages/OpenAttack/utils/zip_downloader.py 

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

with urllib.request.urlopen(remote_url, context=ctx) as fin:
```

## Universal Adversarial Trigger bug
There also seems to be a bug in the original code that is downloaded as a pip package. For some reason pip doesn't download the most recent OpenAttack, so you need to manually integrate this commit
https://github.com/thunlp/OpenAttack/commit/50606c3acd1e1ac192fcc2f80997580f274c5ae2

On Colab its located in
```
/usr/local/lib/python3.8/dist-packages/OpenAttack/attackers/uat/__init__.py
```
## IMPORTANT
Afterwards re-run the import cell!


In [ ]:
# Proceed to attacking (rawr)
print("Initialize some mad hacks")
attacker = OpenAttack.attackers.UATAttacker()
attacker.set_triggers(clsf, devSet, beam_size=3, epoch=2)

Initialize some mad hacks


Epoch 1: 100%|██████████| 900/900 [13:04<00:00,  1.15it/s]


In [ ]:
print("Start some mad hacks")
attack_eval = OpenAttack.AttackEval( attacker, clsf, metrics=[
    OpenAttack.metric.Fluency(),
    OpenAttack.metric.GrammaticalErrors(),
    OpenAttack.metric.EditDistance(),
    OpenAttack.metric.ModificationRate()
] )
attack_eval.eval(devSet, visualize=True, progress_bar=True)